In [2]:
import ollama
import json
import re

def get_customer_decision_optimized(customer_profile: dict, product_name: str) -> dict:
    """
    Optimized simulation with error handling, token limits, and role separation.
    """
    
    # OPTIMIZATION 1: Clean Input
    # Remove 'customer_id' so the model doesn't waste attention on it or get confused
    profile_clean = {k: v for k, v in customer_profile.items() if k != 'customer_id'}
    
    # OPTIMIZATION 2: System vs User Prompt Separation
    # System Prompt: Contains the RULES (Static)
    system_prompt = """
    You are a simulator for a retail bank customer.
    Evaluate offers based strictly on your profile keys (balance, spending, salary, responsiveness, savings_rate).
    
    OUTPUT RULES:
    1. Respond ONLY in valid JSON.
    2. No markdown formatting (no ```json ... ```).
    3. Keys: "decision" (accepted/rejected), "satisfaction_score" (1-10), "reason" (max 10 words).
    """
    
    # User Prompt: Contains the DATA (Dynamic)
    user_prompt = f"""
    MY PROFILE: {json.dumps(profile_clean)}
    OFFER: "{product_name}"
    """

    try:
        response = ollama.chat(
            model='qwen2.5:7b',
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_prompt}
            ],
            format='json',
            options={
                'temperature': 0.1,    # Low creativity for consistency
                'num_predict': 60,     # <--- SPEED HACK: Hard limit on output length
                'num_ctx': 1024        # Limit context window if not needed (saves RAM)
            }
        )

        content = response['message']['content']
        
        # OPTIMIZATION 3: Robust JSON Parsing
        # Strip markdown code blocks if the model adds them despite instructions
        if "```" in content:
            content = re.sub(r"```json|```", "", content).strip()
            
        return json.loads(content)

    except json.JSONDecodeError:
        # Fallback if model returns garbage
        return {"decision": "error", "satisfaction_score": 0, "reason": "json_parse_error"}
    except Exception as e:
        print(f"Error: {e}")
        return {"decision": "error", "satisfaction_score": 0, "reason": "api_error"}

# --- TEST ---
customer_profile = {
    'customer_id': 3,
    'balance': 14269.44,
    'avg_spending': 2087.64,
    'salary': 5126,
    'responsiveness': 0.83,
    'savings_rate': 0.045
}

product_name = "Reminder"

print("⏳ Simulating...")
result = get_customer_decision_optimized(customer_profile, product_name)
print(result)

⏳ Simulating...
{'decision': 'accepted', 'satisfaction_score': 9, 'reason': 'Relevant reminder.'}


In [4]:
# Example usage
customer_profile = {
    'customer_id': 3,
    'balance': 14269.44,
    'avg_spending': 2087.64,
    'salary': 5126,
    'responsiveness': 0.83,
    'savings_rate': 0.045
}

product_name = "Premium Credit Card"

result = get_customer_decision_optimized(customer_profile, product_name)
print("✅ Decision:", result['decision'])
print("Score", result['satisfaction_score'])
print("📝 Reason:", result['reason'])

✅ Decision: accepted
Score 9
📝 Reason: Good balance and savings rate


In [3]:
# Example usage - likely to be rejected
customer_profile = {
    'customer_id': 42,
    'balance': 850.00,
    'avg_spending': 2500.00,
    'salary': 2200,
    'responsiveness': 0.25,
    'savings_rate': 0.005
}

product_name = "Premium Platinum Credit Card with $500 Annual Fee"

result = get_customer_decision_optimized(customer_profile, product_name)
print("✅ Decision:", result['decision'])
print("Score", result['satisfaction_score'])
print("📝 Reason:", result['reason'])

✅ Decision: rejected
Score 3
📝 Reason: High fee, low balance.


In [13]:
customer_profile = {
    'customer_id': 31,
    'balance': 5914,
    'avg_spending': 2763.36,
    'monthly_salary': 3904,
    'savings_rate': 0.028,
    'responsiveness': 0.6
}

product_name = "High Yield Checking Account"

result = get_customer_decision(customer_profile, product_name)
print("✅ Decision:", result['decision'])
print("Score", result['satisfaction_score'])
print("📝 Reason:", result['reason'])

✅ Decision: accepted
Score 7
📝 Reason: Good rate, slight improvement


In [6]:
customer_profile = {
    'customer_id': 42,
    'balance': 850.00,
    'avg_spending': 2500.00,
    'salary': 2200,
    'responsiveness': 0.25,
    'savings_rate': 0.005
}

product_name = "Premium Platinum Credit Card with $500 Annual Fee"

result = get_customer_decision(customer_profile, product_name)
print("✅ Decision:", result['decision'])
print("Score", result['satisfaction_score'])
print("📝 Reason:", result['reason'])

✅ Decision: rejected
Score 2
📝 Reason: Low balance can't afford annual fee.


In [9]:
customer_profile = {
    'customer_id': 26,
    'balance': 17976.57,
    'avg_spending': 6233.3,
    'salary': 6012,
    'savings_rate': 0.019,
    'responsiveness': 0.41
}

product_name = "Premium Platinum Credit Card with $500 Annual Fee"

result = get_customer_decision(customer_profile, product_name)
print("✅ Decision:", result['decision'])
print("Score", result['satisfaction_score'])
print("📝 Reason:", result['reason'])

✅ Decision: rejected
Score 2
📝 Reason: Low balance and savings-focused.


In [ ]:
import ollama
import json
import pandas as pd

def batch_process_customers_single_prompt(customers: list, product_name: str) -> pd.DataFrame:
    prompt = f"""
        You are a simulator for retail bank customers.

        ### THE SCENARIO:
        The bank has offered: "{product_name}" to {len(customers)} customers.

        ### CUSTOMER PROFILES (ARRAY OF {len(customers)} ITEMS):
        {json.dumps(customers, indent=2)}

        ### YOUR TASK:
        For EACH customer, output exactly one object with:
        - customer_id
        - decision ("accepted" or "rejected")
        - satisfaction_score (1-10)
        - reason (max 10 words)

        ### OUTPUT REQUIREMENT:
        Return ONLY valid JSON array with exactly {len(customers)} objects, one per input customer, preserving the same order.
        Example:
        [
        {{
            "customer_id": 1,
            "decision": "accepted",
            "satisfaction_score": 7,
            "reason": "short text"
        }},
        ...
        ]
        """

    response = ollama.chat(
        model='qwen2.5:7b',
        messages=[{'role': 'user', 'content': prompt}],
        format='json',
        options={'temperature': 0.1}
    )

    raw = response['message']['content']
    results = json.loads(raw)
    
    # Unwrap if the model wrapped under "customers"
    if isinstance(results, dict) and "customers" in results:
        results = results["customers"]
    
    # Normalize to list
    if isinstance(results, dict):
        results = [results]
    
    # Validate length
    if not isinstance(results, list) or len(results) != len(customers):
        raise ValueError(f"Model returned {len(results) if isinstance(results, list) else 'non-list'} items, expected {len(customers)}.")
    
    return pd.DataFrame(results)

In [ ]:
# Example: 15 sample customers
customers = [
    # High-value customers
    {'customer_id': 1, 'balance': 25000, 'avg_spending': 3000, 'salary': 7000, 'responsiveness': 0.85, 'savings_rate': 0.08},
    {'customer_id': 4, 'balance': 45000, 'avg_spending': 8000, 'salary': 12000, 'responsiveness': 0.90, 'savings_rate': 0.10},
    
    # Low-income, struggling customers
    {'customer_id': 2, 'balance': 1200, 'avg_spending': 2500, 'salary': 2800, 'responsiveness': 0.30, 'savings_rate': 0.01},
    {'customer_id': 5, 'balance': 500, 'avg_spending': 1800, 'salary': 2000, 'responsiveness': 0.20, 'savings_rate': 0.002},
    
    # Middle-class, moderate savers
    {'customer_id': 3, 'balance': 8500, 'avg_spending': 1500, 'salary': 4500, 'responsiveness': 0.65, 'savings_rate': 0.05},
    {'customer_id': 6, 'balance': 12000, 'avg_spending': 2200, 'salary': 5000, 'responsiveness': 0.55, 'savings_rate': 0.04},
    
    # High spenders, low savers
    {'customer_id': 7, 'balance': 3000, 'avg_spending': 4500, 'salary': 5500, 'responsiveness': 0.70, 'savings_rate': 0.01},
    {'customer_id': 8, 'balance': 15000, 'avg_spending': 6000, 'salary': 7500, 'responsiveness': 0.45, 'savings_rate': 0.015},
    
    # Conservative savers
    {'customer_id': 9, 'balance': 35000, 'avg_spending': 1800, 'salary': 6000, 'responsiveness': 0.75, 'savings_rate': 0.12},
    {'customer_id': 10, 'balance': 50000, 'avg_spending': 2000, 'salary': 8000, 'responsiveness': 0.80, 'savings_rate': 0.15},
    
    # Young professionals
    {'customer_id': 11, 'balance': 5000, 'avg_spending': 3500, 'salary': 4800, 'responsiveness': 0.60, 'savings_rate': 0.03},
    {'customer_id': 12, 'balance': 7500, 'avg_spending': 4200, 'salary': 6200, 'responsiveness': 0.68, 'savings_rate': 0.035},
    
    # Retired/fixed income
    {'customer_id': 13, 'balance': 40000, 'avg_spending': 1500, 'salary': 3500, 'responsiveness': 0.40, 'savings_rate': 0.08},
    {'customer_id': 14, 'balance': 60000, 'avg_spending': 1800, 'salary': 4000, 'responsiveness': 0.35, 'savings_rate': 0.10},
    
    # Financially stressed
    {'customer_id': 15, 'balance': 200, 'avg_spending': 3000, 'salary': 3200, 'responsiveness': 0.15, 'savings_rate': 0.001},
]

product_name = "Premium Platinum Credit Card"

# Run single-prompt batch processing
results_df = batch_process_customers_single_prompt(customers, product_name)
print(results_df.to_string(index=False))

 customer_id decision  satisfaction_score                         reason
           1 rejected                   2        low balance, low salary
           4 accepted                   9      high balance and spending
           2 rejected                   1        low balance, low salary
           5 rejected                   1        low balance, low salary
           3 accepted                   6  moderate balance and spending
           6 accepted                   8  good balance, moderate salary
           7 rejected                   1        low balance, low salary
           8 accepted                   7      good balance and spending
           9 accepted                   9  high balance, moderate salary
          10 accepted                  10 very high balance and spending
          11 rejected                   2        low balance, low salary
          12 accepted                   7  moderate balance and spending
          13 rejected                   1        lo

In [28]:
results_df

,customer_id,decision,satisfaction_score,reason
0,1,rejected,2,"low balance, low salary"
1,4,accepted,9,high balance and spending
2,2,rejected,1,"low balance, low salary"
3,5,rejected,1,"low balance, low salary"
4,3,accepted,6,moderate balance and spending
5,6,accepted,8,"good balance, moderate salary"
6,7,rejected,1,"low balance, low salary"
7,8,accepted,7,good balance and spending
8,9,accepted,9,"high balance, moderate salary"
9,10,accepted,10,very high balance and spending


In [37]:
len(results_df[results_df['decision'] == 'accepted']) / len(results_df)

0.5333333333333333